In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip install face_alignment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import face_alignment
import numpy as np
import os
from PIL import Image
from skimage import io
import torch
import torchvision
import torch.nn as nn
import matplotlib.pyplot as plt

import copy

In [ ]:
def image_loader(image_name):
    image = Image.open(image_name).convert('RGB').resize((512,512))
    #display(image)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    #print(image.shape)
    return image.to(device, torch.float)

In [ ]:
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False, face_detector='sfd')

faces = {}

labels = ["surprise", "sad", "neutral", "happy", "disgust"]

l_convert = {
    "surprise": 0,
    "sad": 1,
    "neutral": 2,
    "happy": 3,
    "disgust": 4
}

for folder in labels:
  directory = os.fsencode("/content/drive/MyDrive/Faces/" + folder + "/")
  faces[folder] = []
  for f in os.listdir(directory):
      filename = os.fsdecode(f)
      img = io.imread("/content/drive/MyDrive/Faces/" + folder + "/" + filename)
      preds = fa.get_landmarks(img)
      if preds is not None:
        faces[folder].append(torch.Tensor(preds[0]).flatten())


imgs = []
labels = []

for label, coordss in faces.items():
  for coords in coordss:
    imgs.append(coords)
    l_zero = torch.zeros(5)
    l_zero[l_convert[label]] = 1
    labels.append(l_zero)

/usr/local/lib/python3.7/dist-packages/face_alignment/api.py:144: UserWarning: No faces were detected.
  warnings.warn("No faces were detected.")


In [ ]:
print(imgs[0].shape)

torch.Size([136])


In [ ]:
class LandmarkDataset(torch.utils.data.Dataset):
    def __init__(self):
        pass

    def __len__(self):
        return len(imgs)

    def __getitem__(self, idx):
        return imgs[idx], labels[idx]

In [ ]:
train_data = LandmarkDataset()
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(136, 100), # the "connections between" layers
            nn.ReLU(),
            nn.Linear(100, 70),
            nn.ReLU(),
            nn.Linear(70, 50),
            nn.ReLU(),
            nn.Linear(50, 40),
            nn.ReLU(),
            nn.Linear(40, 5)
        )

    def forward(self, x):
        flattened = self.flatten(x)
        prediction = self.layers(flattened)
        return prediction

In [ ]:
model = NeuralNet()

In [ ]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.0001
epochs = 700
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train(dataloader, model, loss_fn, optimizer, epochs):
  size = len(dataloader.dataset)
  # epoch = one pass over the dataset
  for epoch in range(epochs):
    print("epoch: " + str(epoch))
    # since batch = 1, one batch is one example from the training set
    for batch, (X, y) in enumerate(dataloader):

      # compute loss
      pred = model(X)
      #print("prediction: ", pred)
      #print("label:", y)
      loss = loss_fn(pred, y)

      # backpropagation
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if batch % 10 == 0:
        loss = round(float(loss.item()), 3)
        current_iter = batch * len(X)
        print("loss: " + str(loss) + " [" + str(current_iter) + "/" + str(size) + "]")
        #print(100*(epoch + (current_iter/size)))

In [ ]:
train(train_dataloader, model, loss_fn, optimizer, epochs)

epoch: 0
loss: 1.631 [0/679]
loss: 1.46 [160/679]
loss: 1.602 [320/679]
loss: 1.659 [480/679]
loss: 1.458 [640/679]
epoch: 1
loss: 1.69 [0/679]
loss: 1.55 [160/679]
loss: 1.323 [320/679]
loss: 1.39 [480/679]
loss: 1.42 [640/679]
epoch: 2
loss: 1.453 [0/679]
loss: 1.422 [160/679]
loss: 1.507 [320/679]
loss: 1.385 [480/679]
loss: 1.564 [640/679]
epoch: 3
loss: 1.479 [0/679]
loss: 1.361 [160/679]
loss: 1.521 [320/679]
loss: 1.409 [480/679]
loss: 1.573 [640/679]
epoch: 4
loss: 1.313 [0/679]
loss: 1.421 [160/679]
loss: 1.542 [320/679]
loss: 1.606 [480/679]
loss: 1.611 [640/679]
epoch: 5
loss: 1.665 [0/679]
loss: 1.563 [160/679]
loss: 1.424 [320/679]
loss: 1.593 [480/679]
loss: 1.585 [640/679]
epoch: 6
loss: 1.416 [0/679]
loss: 1.512 [160/679]
loss: 1.594 [320/679]
loss: 1.449 [480/679]
loss: 1.645 [640/679]
epoch: 7
loss: 1.507 [0/679]
loss: 1.504 [160/679]
loss: 1.534 [320/679]
loss: 1.43 [480/679]
loss: 1.58 [640/679]
epoch: 8
loss: 1.461 [0/679]
loss: 1.35 [160/679]
loss: 1.639 [320/679]

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  loss_sum = 0
  for X, y in dataloader:
    pred = model(X)
    #print("prediction: ", pred)
    #print("label:", y)
    loss = loss_fn(pred, y)
    #print(loss)
    loss_sum += loss

  avg = loss_sum/size
  print("avg loss: " + str(avg))

In [ ]:
test(test_dataloader, model, loss_fn)

avg loss: tensor(0.5191, grad_fn=<DivBackward0>)


In [ ]:
torch.save(model, "hopefully_better_test_model_2.pth")